# Filtering, Select, Ordering

Now that you know what LINQ is, let's practise the three most common operations you'll use with in‑memory collections that implement `IEnumerable<T>`.

- **Filtering** keeps only the items you want.
- **Selecting** (projection) transforms each item into a new shape.
- **Ordering** sorts the results.

## Sample data

In [ ]:
// Program.cs (top-level)
var products = new List<Product>
{
    new Product { Name = "Mug",        Category = "Kitchen", Price = 6.50m,  InStock = true  },
    new Product { Name = "Notebook",   Category = "Stationery", Price = 3.20m, InStock = true  },
    new Product { Name = "Pen",        Category = "Stationery", Price = 1.80m, InStock = false },
    new Product { Name = "Tea Kettle", Category = "Kitchen", Price = 24.90m, InStock = true  },
    new Product { Name = "Lamp",       Category = "Home", Price = 18.00m, InStock = false }
};

// Product.cs
public class Product
{
    public string Name { get; set; } = string.Empty;
    public string Category { get; set; } = string.Empty;
    public decimal Price { get; set; }
    public bool InStock { get; set; }
}

## Filtering with `Where`

In [ ]:
// Program.cs (top-level)
var products = new List<Product>
{
    new Product { Name = "Mug",        Category = "Kitchen", Price = 6.50m,  InStock = true  },
    new Product { Name = "Notebook",   Category = "Stationery", Price = 3.20m, InStock = true  },
    new Product { Name = "Pen",        Category = "Stationery", Price = 1.80m, InStock = false },
    new Product { Name = "Tea Kettle", Category = "Kitchen", Price = 24.90m, InStock = true  },
    new Product { Name = "Lamp",       Category = "Home", Price = 18.00m, InStock = false }
};
// In-stock stationery only
var inStockStationery = products
    .Where(p => p.Category == "Stationery" && p.InStock);

foreach (var p in inStockStationery)
{
    Console.WriteLine($"{p.Name} (£{p.Price:0.00})");
}

// Product.cs
public class Product
{
    public string Name { get; set; } = string.Empty;
    public string Category { get; set; } = string.Empty;
    public decimal Price { get; set; }
    public bool InStock { get; set; }
}


**Tip:** Combine conditions with `&&` (and), `||` (or). Use `StringComparison` if you need case‑insensitive checks.

## Selecting (projection) with `Select`

In [ ]:
// Program.cs (top-level)
var products = new List<Product>
{
    new Product { Name = "Mug",        Category = "Kitchen", Price = 6.50m,  InStock = true  },
    new Product { Name = "Notebook",   Category = "Stationery", Price = 3.20m, InStock = true  },
    new Product { Name = "Pen",        Category = "Stationery", Price = 1.80m, InStock = false },
    new Product { Name = "Tea Kettle", Category = "Kitchen", Price = 24.90m, InStock = true  },
    new Product { Name = "Lamp",       Category = "Home", Price = 18.00m, InStock = false }
};
// Project to just names
var names = products.Select(p => p.Name);
foreach (var name in names) Console.WriteLine(name);

// Project to an anonymous type (shaping data for the UI)
var display = products.Select(p => new { p.Name, PriceWithVat = p.Price * 1.20m });
foreach (var d in display) Console.WriteLine($"{d.Name} → £{d.PriceWithVat:0.00}");

// Project to a DTO/record if you prefer strong types
var cards = products.Select(p => new ProductCard(p.Name, p.Price));
foreach (var c in cards) Console.WriteLine($"{c.Title} (£{c.Price:0.00})");

// Product.cs
public class Product
{
    public string Name { get; set; } = string.Empty;
    public string Category { get; set; } = string.Empty;
    public decimal Price { get; set; }
    public bool InStock { get; set; }
}
// ProductCard.cs
public record ProductCard(string Title, decimal Price);

## Ordering with `OrderBy`, `ThenBy`, and descending variants

In [ ]:
// In Program.cs
// Order by price ascending
var byPrice = products.OrderBy(p => p.Price);
foreach (var p in byPrice) Console.WriteLine($"{p.Name} £{p.Price:0.00}");

// Order by category, then price within each category
var byCategoryThenPrice = products
    .OrderBy(p => p.Category)
    .ThenBy(p => p.Price);
foreach (var p in byCategoryThenPrice) Console.WriteLine($"{p.Category} – {p.Name} £{p.Price:0.00}");

// Highest price first
var expensiveFirst = products.OrderByDescending(p => p.Price);
foreach (var p in expensiveFirst) Console.WriteLine($"£{p.Price:0.00} – {p.Name}");

**Tip:** Ordering does not change the original list; it returns a new ordered sequence. If you need a materialised list, call `.ToList()` at the end.

## Chaining operations

You'll often chain filter → select → order for clear intent.

In [ ]:
// Program.cs (top-level)
var products = new List<Product>
{
    new Product { Name = "Mug",        Category = "Kitchen", Price = 6.50m,  InStock = true  },
    new Product { Name = "Notebook",   Category = "Stationery", Price = 3.20m, InStock = true  },
    new Product { Name = "Pen",        Category = "Stationery", Price = 1.80m, InStock = false },
    new Product { Name = "Tea Kettle", Category = "Kitchen", Price = 24.90m, InStock = true  },
    new Product { Name = "Lamp",       Category = "Home", Price = 18.00m, InStock = false }
};

// In Program.cs
var topKitchenNames = products
    .Where(p => p.Category == "Kitchen" && p.InStock)
    .OrderByDescending(p => p.Price)
    .Select(p => p.Name)
    .Take(3)          // optional: limit the result
    .ToList();        // materialise to a list if needed but you lose lazy evaluation

foreach (var name in topKitchenNames) Console.WriteLine(name);

// Product.cs
public class Product
{
    public string Name { get; set; } = string.Empty;
    public string Category { get; set; } = string.Empty;
    public decimal Price { get; set; }
    public bool InStock { get; set; }
}

## Summary

- **Filter** with `Where`, **shape** with `Select`, **sort** with `OrderBy/ThenBy`.
- LINQ works on any `IEnumerable<T>` (arrays, lists, etc.).
- Method chains read from left to right, making intent clear.
- Materialise with `.ToList()` only when you need a list; otherwise keep it as an `IEnumerable<T>`.